In [1]:
import numpy as np
import pandas as pd
import os, re, shutil

In [2]:
import sys 
sys.path.append('Shell_Code_Modified/')
import Farm_Evaluator_Vec


In [39]:
#initialize all the variables
# wind farm size, cells
cols_cells = 80  # number of cells each row
rows_cells = 80  # number of cells each column
cell_width = 50 # unit (TBD)--> have to be computed.
wt_N=50  #number of turbines

cand_per_run=10

folderName="Ver1_N/naveen_data"
ExtName= (folderName+"/results/sg")

best_layout_prefix="sugga_best_layouts_N"
best_energy_prefix="sugga_eta_N"
default_location='../data/Shell_Hackathon_Dataset/'
Wind_data_year=[2017,2015,2014,2013,2009,2008,2007]


output_folder_location="Ver1_N/results"

In [4]:
def convert_xy_pos(pop,rows,cell_width,N_tur):
    xy_position = np.zeros((N_tur,2), dtype=np.float32)
    curr_tur=0
    for ind in range(rows * rows):
        if pop[ind] == 1:
            r_i = np.floor(ind / rows)
            c_i = np.floor(ind - r_i * rows)
            xy_position[curr_tur,0] = c_i * cell_width + cell_width/2
            xy_position[curr_tur,1] = r_i * cell_width + cell_width/2
            curr_tur +=1
    return (xy_position)
            

In [96]:

# Create regex with prefix + number + extension.
fileRegex = re.compile(r'(%s)\d' % best_layout_prefix)
    
# Make a list to contain the file with prefix.
fileNames = []
for filename in os.listdir(ExtName):
    #print(filename)
    if re.search(fileRegex, filename):
        fileNames.append(filename)
# Make sure the fileName in the list have a right order.

run_results_dict={}

result_summary_df=pd.DataFrame() #dataframe to analyze results
result_list_all={}  #complete summary with xy coordinates position.
dict_index=0

result_layout_all=None
best_eta_layout_all=None

for curr_run_filename in fileNames:
    print("Processing File:"+curr_run_filename)
    filePath=ExtName+"/"+curr_run_filename
    result_layout=np.genfromtxt(filePath, delimiter="  ", dtype=np.int32)
#     print(result_layout_all.shape)
#     print(result_layout.shape)

    
    best_eta_filePath=curr_run_filename.replace(best_layout_prefix,best_energy_prefix)
    best_eta_filePath = ExtName+"/"+best_eta_filePath
    best_eta_layout=np.genfromtxt(best_eta_filePath, delimiter="  ")
#     print(best_eta_layout.shape)
#     print(best_eta_layout_all.shape)
    if result_layout_all is None:
        result_layout_all=result_layout
        best_eta_layout_all=best_eta_layout
    else:
        result_layout_all=np.vstack((result_layout,result_layout_all))
        best_eta_layout_all=np.append(best_eta_layout,best_eta_layout_all,axis=0)

print(result_layout_all.shape)
print(best_eta_layout_all.shape)

best_run_sorts=np.argsort(best_eta_layout_all)
#   print(best_eta_layout)
total_cands=best_run_sorts.shape[0]
dict_index=0
for curr_cand in range(cand_per_run):

    curr_run_results={}
    
    tur_xy_coord=convert_xy_pos(result_layout_all[best_run_sorts[total_cands-1-curr_cand]],rows_cells,cell_width,wt_N)

    #         print(tur_xy_coord.shape)
    for curr_wind_inst in Wind_data_year:
        wind_inst_freq_file=default_location+"Wind_Data/wind_data_"+str(curr_wind_inst)+".csv"
        total_energy, turbine_power=Farm_Evaluator_Vec.get_AEP_results(default_location=default_location,turb_coords=tur_xy_coord,wind_inst_freq_file=wind_inst_freq_file)
        curr_run_results[str(curr_wind_inst)]=total_energy
#             print("Year: "+str(curr_wind_inst)+" Energy: "+ str(total_energy))
#         print(curr_run_results)
    curr_run_results['dict_index']=dict_index
#           print(curr_run_results)
#         print(result_summary_df)

    result_summary_df=result_summary_df.append(pd.DataFrame.from_records([curr_run_results]),ignore_index = True)
    curr_run_results['XY_coords']=tur_xy_coord
    result_list_all[dict_index]=curr_run_results
    dict_index +=1


Processing File:sugga_best_layouts_N50_0_20201010215944.dat
Processing File:sugga_best_layouts_N50_1_20201010224127.dat
(200, 6400)
(200,)


In [108]:
#assign overall score based on year. 
result_summary_df['best_score']=(result_summary_df['2007']+result_summary_df['2008']+result_summary_df['2009']+result_summary_df['2013']+result_summary_df['2014']+result_summary_df['2015']+result_summary_df['2017'])/7
result_summary_df.sort_values(by=['best_score'],inplace=True,ascending=False)
result_summary_df['file_name']=np.arange(0,result_summary_df.shape[0])
result_summary_df.head()
#can be used to select top N (future in case needed)

,2007,2008,2009,2013,2014,2015,2017,dict_index,fileName,best_score,file_name
5,523.499457,517.64541,473.791977,489.460255,464.660216,577.365751,563.203332,5,sugga_best_layouts_N50_1_20201010224127.dat,515.660914,0
6,523.499457,517.64541,473.791977,489.460255,464.660216,577.365751,563.203332,6,sugga_best_layouts_N50_1_20201010224127.dat,515.660914,1
7,523.499457,517.64541,473.791977,489.460255,464.660216,577.365751,563.203332,7,sugga_best_layouts_N50_1_20201010224127.dat,515.660914,2
8,523.499457,517.64541,473.791977,489.460255,464.660216,577.365751,563.203332,8,sugga_best_layouts_N50_1_20201010224127.dat,515.660914,3
9,523.499457,517.64541,473.791977,489.460255,464.660216,577.365751,563.203332,9,sugga_best_layouts_N50_1_20201010224127.dat,515.660914,4


In [110]:
#write outputs
os.makedirs(output_folder_location, exist_ok=True)

for index, curr_row in result_summary_df.iterrows():
    curr_dictIndex=curr_row['dict_index']
    curr_result_xy=pd.DataFrame(result_list_all[curr_dictIndex]['XY_coords'])
    curr_result_xy.columns=["x","y"]
    curr_result_xy.to_csv(output_folder_location+"/best"+str(curr_row['file_name'])+".csv",index=False)
    outfile_index +=1

result_summary_df.to_csv(output_folder_location+"/result_summary.csv",index=False)
print("All Files are stored at location: "+output_folder_location)

All Files are stored at location: Ver1_N/results
